In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from lxml import etree

In [2]:
plants_list_url = "https://cb.imsc.res.in/imppat/#browse"

In [3]:
plants_list_page = requests.get(plants_list_url)

In [4]:
plants_list_soup = bs(plants_list_page.content, 'html.parser')
plants_list = plants_list_soup.find('label', string=re.compile('Choose an Indian medicinal plant:')).find_next_sibling('select').find_all('option')
plants_list = plants_list[1:]
plants_list = [plant.text for plant in plants_list if plant.text != '']

In [5]:
therapeutic_link = "https://cb.imsc.res.in/imppat/therapeutics/"

In [9]:
len(plants_list)

4010

In [7]:
required_fields = ['kingdom', 'family', 'group', 'common name', 'synonymous names', 'system of medicine', 'iucn red list category']
tables = []
for index, plant in enumerate(plants_list):
    print(f"Processing {index} of {len(plants_list)} ({plant})")
    page = requests.get(therapeutic_link + plant)
    soup = bs(page.content, 'html.parser')
    html_parser = etree.HTMLParser()
    tree = etree.HTML(str(soup))
    
    # plant details
    ele = tree.xpath('/html/body/div[2]/div[1]/div[1]/strong')
    fields = [e.text.strip().split(':')[0] for e in ele]
    fields = [field.lower() for field in fields if field.lower() in required_fields]

    values = tree.xpath('/html/body/div[2]/div[1]/div[1]/text()')
    values = [value.strip() for value in values if value.strip() != '']
    
    # plant therapeutic details
    table = pd.read_html(page.content)
    df = table[0]

    for _idx, field in enumerate(fields):
        df[field] = values[_idx]
    tables.append(df)

Processing 0 of 4010 (Abelmoschus esculentus)
Processing 1 of 4010 (Abelmoschus ficulneus)
Processing 2 of 4010 (Abelmoschus manihot)
Processing 3 of 4010 (Abelmoschus moschatus)
Processing 4 of 4010 (Abies alba)
Processing 5 of 4010 (Abies balsamea)
Processing 6 of 4010 (Abies grandis)
Processing 7 of 4010 (Abies nephrolepis)
Processing 8 of 4010 (Abies pindrow)
Processing 9 of 4010 (Abies pinsapo)
Processing 10 of 4010 (Abies spectabilis)
Processing 11 of 4010 (Abroma augusta)
Processing 12 of 4010 (Abrus precatorius)
Processing 13 of 4010 (Abutilon grandifolium)
Processing 14 of 4010 (Abutilon hirtum)
Processing 15 of 4010 (Abutilon indicum)
Processing 16 of 4010 (Abutilon pannosum)
Processing 17 of 4010 (Abutilon ramosum)
Processing 18 of 4010 (Abutilon theophrasti)
Processing 19 of 4010 (Acacia aroma)
Processing 20 of 4010 (Acacia auriculiformis)
Processing 21 of 4010 (Acacia caesia)
Processing 22 of 4010 (Acacia catechu)
Processing 23 of 4010 (Acacia caven)
Processing 24 of 4010 

In [8]:
dfs = pd.concat(tables)

In [9]:
dfs.to_csv('datasets/imppat1.csv', index=False)